In [ ]:
from __future__ import unicode_literals # 이게 뭘까
import requests
import pymongo
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# from scrapy.http import TextResponse
import urllib.request
import urllib
import getpass
import time
import re
# from google.cloud import vision
# from google.cloud.vision import types
import os
import io
from os import walk
from bs4 import BeautifulSoup

In [ ]:
!cd

In [26]:
if not os.path.exists('./thumb_detail'):
    os.mkdir('./thumb_detail')

webtoon_info = []
total_df_info = []
total_img_count = 0

domain_url = 'https://comic.naver.com'
main_url = 'https://comic.naver.com/webtoon/weekday.nhn'

html = requests.get(main_url).text
soup = BeautifulSoup(html, 'html.parser')
webtoon_area = soup.select('#content > div.list_area.daily_all > div.col > div > ul > li > div.thumb > a')

for each_webtoon in webtoon_area:
    webtoon_last_page = domain_url + each_webtoon['href'] + '&page={}'.format(99999)
    html = requests.get(webtoon_last_page).text
    soup = BeautifulSoup(html, 'html.parser')
    last_page_num = int(soup.select('#content > div.paginate > div > strong.page')[0].find('em').text)
    
    title = soup.select('.comicinfo h2')[0].text.split('\n')[1] 
#     title = ', '.join([each_ele.strip() for each_ele in soup.select('.comicinfo h2')[0].text.split('\n') if each_ele != ''])
    title = ', '.join(['&'.join([each_text.strip() for each_text in each_text.split('/')]) for each_text in title.split(',')])
    title = re.sub('[-=+#/\?:^$.@*\"※~%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', title)
#     print(title)
    
    author = soup.select('.comicinfo h2')[0].text.split('\n')[2] 
#     author = ', '.join([each_ele.strip() for each_ele in soup.select('.comicinfo h2')[0].text.split('\n') if each_ele != ''])
    author = ', '.join([','.join([each_text.strip() for each_text in each_text.split('/')]) for each_text in author.split(',')])
    author = re.sub('[-=+#/\?:^$.@*\"※~%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', author)
#     print(author)


    genre = soup.select('#content > div.comicinfo > div.detail > p.detail_info > span.genre')[0].text
    content = soup.select('#content > div.comicinfo > div.detail > p:nth-child(2)')[0].text
    age = soup.select('#content > div.comicinfo > div.detail > p.detail_info > span.age')[0].text

    webtoon_info.append({'title': title, 'author': author, 'genre': genre, 'content': content, 'age': age})

    for page_num in range(1, last_page_num + 1):
        webtoon_url = domain_url + each_webtoon['href'] + '&page={}'.format(page_num)
        html = requests.get(webtoon_url).text
        soup = BeautifulSoup(html, 'html.parser')

        if not os.path.exists('./thumb_detail/{}'.format(author)):
            os.mkdir('./thumb_detail/{}'.format(author))

        thumbnail_area = soup.find('table', 'viewList')
        thumbnail_img_list = thumbnail_area.select('td > a > img')[1:]
        
#         for each_img_tag in thumbnail_img_list:
#             print(re.sub('[^0-9a-zA-Zㄱ-힗]','',each_img_tag['alt'].split('/')[0]))


        for each_img_tag in thumbnail_img_list:
            urllib.request.urlretrieve(each_img_tag['src'], './thumb_detail/{}/{}.png'.format(author, title + '_' + re.sub('[\\/:*?"<>|]','',str(each_img_tag['alt'].split('/')[0]))))
#             urllib.request.urlretrieve(each_img_tag['src'], './total_img/{}.png'.format(author + '_' + '&'.join(each_img_tag['alt'].split('/')[0])))
              total_df_info.append({'title': title, 'author': author, 'genre': genre, 'content': content, 'age': age})
              total_img_count += 1
        
    print(title+' done')
            

df = pd.DataFrame(webtoon_info, columns=['title', 'author', 'genre', 'content', 'age'])
total_df = pd.DataFrame(total_df_info, columns=['title', 'author', 'genre', 'content', 'age'])
df.to_csv('webtoon_info.csv')
total_df.to_csv('total_img_info.csv')

참교육 done
신의 탑 done
뷰티풀 군바리 done
소녀의 세계 done
팔이피플 done
장씨세가 호위무사 done
백수세끼 done
파이게임 done
윈드브레이커휴재 done
앵무살수 done
홍천기 done
만렙돌파 done
삼매경 done
더블클릭 done
잔불의 기사 done
요리GO done
말박왕 done
똑 닮은 딸 done
유일무이 로맨스 done
히어로메이커 done
바퀴 done
약초마을 연쇄살초사건 done
물어보는 사이 done
오늘의 순정망화 done
결혼생활 그림일기 done
ㅋㄷㅋㄷ만화 done
꼬리잡기 done
평범한 8반 done
아는 여자애 done
꿈의 기업 done
수영만화일기 done
장난감 done
황제와의 하룻밤 done
아, 쫌 참으세요 영주님 done
칼가는 소녀휴재 done
오빠집이 비어서 done
순정말고 순종 done
영앤리치가 아니야컷툰 done
최후의 금빛아이 done
와이키키 뱀파이어 done
하루의 하루 done
야생천사 보호구역 done
모스크바의 여명 done
착한건 돈이된다 done
사랑의 헌옷수거함 done
선배, 그 립스틱 바르지 마요 done
또다시, 계약 부부 done
왕따협상 done
원하는 건 너 하나 done
백호랑 done
이중첩자 done
라서드 done
이제야 연애 done
마지막 지수 done
드로잉 레시피 done
살아간다 done
바로 보지 않는 done
사막에 핀 달 done
이탄국의 자청비 done
모락모락 왕세자님 done
트리거 done
그림자 신부 done
중독연구소 done
개밥 먹는 남자 done
보살님이 캐리해 done
오로지 오로라 done
기사님을 지켜줘 done
헬로맨스휴재 done
여신강림 done
2021 최애캐 안녕, 잘 지내니컷툰 done
용사가 돌아왔다 done
한림체육관 done
덴큐 done
사신소년 done
하루만 네가 되고 싶어 done
엽총소년 done
중증외상센터  골든 아워 done
랜덤채팅의 그녀 done
신도림 done
니나

In [22]:
total_img_count

163